In [ ]:
import numpy as np
import pylab as plt
import itertools as it
import re

In [ ]:
# Set random-number seed.

rng = np.random.default_rng(17)

In [ ]:
# Make 3-point point clouds.

N = 100
d = 2
xs = rng.normal(size=(N, 3, d))
xs += 7. * rng.normal(size=(N, d))[:, None, :]

In [ ]:
# Compute areas, or really squared-areas of the triangles.

def area(xs):
    """
    ## comments:
    - Computes the area with the absolute value of a cross product.
    
    ## bugs:
    - Untested; unchecked!
    """
    return 0.5 * np.abs((xs[:, 2, 0] - xs[:, 0, 0]) * # x3 - x1
                        (xs[:, 1, 1] - xs[:, 0, 1]) - # y2 - y1
                        (xs[:, 2, 1] - xs[:, 0, 1]) * # y3 - y1
                        (xs[:, 1, 0] - xs[:, 0, 0]))  # x2 - x1

ys = area(xs) ** 2
print(ys.shape)

In [ ]:
# Plot one data point.

j = 51
plt.scatter(xs[j, :, 0], xs[j, :, 1])
plt.title("triangle {}: area {}".format(j, np.sqrt(ys[j])))
plt.axis("equal")

In [ ]:
# Build list of all possible polynomial expressions that DeepSets could make, plus names.

def write_exponent(variable, power):
    if power == 0:
        return None
    if power == 1:
        return variable
    return variable + '^{}'.format(power)

def polynomial_terms(xs, max_degree=4):
    """
    ## bugs:
    - `np.concatenate` is slow?
    """
    N, three, d = xs.shape
    assert three == 3
    assert d == 2
    X = [[np.ones(N)]]
    names = ["1"]
    for degree in range(1, max_degree+1):
        thisX = []
        name = []
        for yexp in range(degree+1):
            xexp = degree - yexp
            thisX.append(np.sum(xs[:, :, 0] ** xexp *
                                xs[:, :, 1] ** yexp, axis=1))
            tmp_names = (write_exponent(v, p) for v, p in zip(['x_i', 'y_i'], [xexp, yexp]))
            tmp_names = list(filter(lambda item: item is not None, tmp_names))
            name.append('(\sum_{{i=1}}^3 {})'.format('\,'.join(tmp_names)))
        X.append(thisX)
        names.append(name)
    return X, names

terms, term_names = polynomial_terms(xs)
print(len(terms))
print(list(len(term) for term in terms))
print(list(name for name in term_names))

In [ ]:
# Find all ordered lists of positive integers that add up to 4.

def integer_combinations(n, s=None, slist=None):
    if slist is None:
        slist = []
    if s is None:
        ss = 0
        ms = 1
        s = []
    else:
        ss = np.sum(s)
        ms = np.max(s)
    if ss >= n:
        slist += [s]
    for j in range(ms, n - ss + 1):
        slist = integer_combinations(n, s + [j], slist)
    return slist

combinations = integer_combinations(4, None)
print(combinations)

In [ ]:
# Reformat the order lists to something useful for the next stage.

def count_integers(s, n):
    counts = np.zeros(n+1).astype(int)
    for i in range(n+1):
        counts[i] = np.sum(sum((j == i) for j in s))
    return counts

counts = list(count_integers(combo, 4) for combo in combinations)
print(counts)

In [ ]:
# Find all possible polynomial terms that are 4th order and consistent with DeepSets.

def design_matrix_columns_and_names(terms, term_names, c, n):
    """
    ## bugs:
    - badly repeated code because I suck
    """
    names = []
    columns = []
    foos = it.product(*(it.combinations_with_replacement(term_names[i], c[i]) for i in range(1, n+1)))
    bars = it.product(*(it.combinations_with_replacement(terms[i], c[i]) for i in range(1, n+1)))
    for foo in foos:
        name = ''
        for f in foo:
            if f != ():
                f = list(f)
                # Now search for and simplify some squares
                for j in range(len(f) - 1):
                    if f[j] == f[j + 1]:
                        f[j] += '^2'
                        f[j + 1] = None
                f = list(filter(lambda item: item is not None, f))
                name += '\,'+'\,'.join(f)
        names += [name, ]
    for bar in bars:
        col = 1.
        for f in bar:
            if f != []:
                col *= np.product(np.array(f), axis=0)
        columns += [col, ]
    return columns, names

X = []
column_names = []
for c in counts:
    cs, ns = design_matrix_columns_and_names(terms, term_names, c, 4)
    X += cs
    column_names += ns
X = np.array(X).T
print(len(column_names), column_names)
print(X.shape)

In [ ]:
# Simplify column names that are repetitive.

for j in range(len(column_names)):
    foo = column_names[j].split(')\\,(')
    for k in range(len(foo) - 1):
        if foo[k] + ')' == '(' + foo[k + 1]:
            foo[k] += ")^2"
            foo[k+1] = None
    foo = list(filter(lambda item: item is not None, foo))
    column_names[j] = ')\,('.join(foo)

print(column_names)

In [ ]:
# Perform the regression and show the learned weights.

beta_hat, _, _, _ = np.linalg.lstsq(X, ys, rcond=None)
print(beta_hat)

In [ ]:
# Test that the solution is exact.

print(np.sqrt(np.mean((ys - X @ beta_hat) ** 2)))
print(np.max(np.abs(ys - X @ beta_hat)))
print(np.allclose(ys, X @ beta_hat))

In [ ]:
# Find the non-zero terms, by a hack.

nonzero = np.arange(len(beta_hat)).astype(int)[np.abs(beta_hat) > 1.e-4] # MAGIC
print(len(nonzero))
print(beta_hat[nonzero])

In [ ]:
# Now do rational number stuff.

magic = 368
print(beta_hat[nonzero] * magic) # Check out this MAGIC!

In [ ]:
# Force the answer to be explicitly rational with the MAGIC and re-check the answer.

beta_hat = np.round(beta_hat * magic) / magic
print(beta_hat)
print(np.sqrt(np.mean((ys - X @ beta_hat) ** 2)))
print(np.max(np.abs(ys - X @ beta_hat)))
print(np.allclose(ys, X @ beta_hat))

In [ ]:
# Output the answer in proper LaTeX.

sgn = ["{:+f}".format(b)[0] for b in beta_hat]
print("\\begin{align}")
print("A^2 = &")
for i,j in enumerate(nonzero[::-1]):
    suffix = ''
    if i % 2 == 1:
        suffix = '\\nonumber \\\\ &'
    if i == 8:
        suffix = ''
    print("{}\\frac{{{:3.0f}}}{{{}}}{} {}".format(sgn[j], np.abs(beta_hat[j]) * magic,
                                                  magic, column_names[j], suffix))
print("\\end{align}")